In [2]:
import torch
import numpy as np
from sympy import shape
from torch.ao.quantization.backend_config.native import weight_only_quint8_dtype_config

# 创建一个零维张量
tensor0d = torch.tensor(1)
# 创建一个一维张量
tensor1d = torch.tensor([1,2,3])
# 创建一个二维张量
tensor2d = torch.tensor([[1,2],
                         [3,4]])
# 创建一个三维张量
tensor3d = torch.tensor([[[1,2],[3,4]],
                         [[5,6],[7,8]]])
ary3d = np.array([[[1,2],[3,4]],
                         [[5,6],[7,8]]])

tensor3d_2 = torch.tensor(ary3d)  # 复制 NumPy 数组
tensor3d_3 = torch.from_numpy(ary3d)   # 与 NumPy 数组共享内存

print(tensor3d_2, tensor3d_3)

tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]]) tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])


In [3]:
ary3d[0,0,0] = 999
print(tensor2d)

tensor([[1, 2],
        [3, 4]])


In [4]:
print(tensor3d_3)

tensor([[[999,   2],
         [  3,   4]],

        [[  5,   6],
         [  7,   8]]])


In [5]:
tensorId = torch.tensor([1,2,3])
print(tensorId.dtype)

torch.int64


In [6]:
floatvec = torch.tensor([1.0,2.0,3.0])
print(floatvec.dtype)

torch.float32


In [7]:
floatvec = tensorId.to(torch.float)
print(floatvec.dtype)

torch.float32


In [8]:
tensor2d = torch.tensor([[1,2,3],[3,4,5]])
tensor2d

tensor([[1, 2, 3],
        [3, 4, 5]])

In [9]:
tensor2d.shape

torch.Size([2, 3])

In [10]:
tensor2d.reshape([3,2])

tensor([[1, 2],
        [3, 3],
        [4, 5]])

In [11]:
tensor2d.view(3,2)

tensor([[1, 2],
        [3, 3],
        [4, 5]])

In [12]:
tensor2d.T

tensor([[1, 3],
        [2, 4],
        [3, 5]])

In [13]:
tensor2d.matmul(tensor2d.T)

tensor([[14, 26],
        [26, 50]])

In [14]:
tensor2d @ tensor2d.T

tensor([[14, 26],
        [26, 50]])

In [15]:
# 将模型视为计算图
import torch.nn.functional as F

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2])
b = torch.tensor([0.0])

z = x1 * w1 + b
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a,y)
print(loss)

tensor(0.0852)


In [16]:
# A.4 轻松实现自动微分
import torch.nn.functional as F
from torch.autograd import grad, backward

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad = True)
b = torch.tensor([0.0], requires_grad = True)

z = x1 * w1 + b
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a, y)

gard_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b = grad(loss, b, retain_graph=True)

print(gard_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [17]:
loss.backward()

print(w1.grad)
print(b.grad)

tensor([-0.0898])
tensor([-0.0817])


In [18]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            torch.nn.Linear(20, num_outputs),
        )
    def forward(self, x):
        logits = self.layers(x)
        return logits

In [19]:
model = NeuralNetwork(50, 3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [20]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("total number of parameters: {}".format(num_params))

total number of parameters: 2213


In [21]:
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0260,  0.0368,  0.1279,  ..., -0.1141,  0.1387,  0.0017],
        [-0.0841,  0.0257, -0.0987,  ...,  0.0104,  0.1334, -0.0606],
        [-0.0807,  0.1149, -0.0630,  ..., -0.0830, -0.0037,  0.0851],
        ...,
        [ 0.1366, -0.1074,  0.1048,  ...,  0.0791, -0.0224,  0.0099],
        [ 0.1351, -0.1118,  0.1177,  ..., -0.0460, -0.1022, -0.0100],
        [ 0.0517,  0.0669,  0.1264,  ...,  0.1149, -0.0658,  0.0326]],
       requires_grad=True)


In [22]:
torch.manual_seed(123)

model = NeuralNetwork(50, 3)
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)


In [23]:
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [24]:
torch.manual_seed(123)

X = torch.rand((1,50))
out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)


In [25]:
with torch.no_grad():
    out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]])


In [26]:
with torch.no_grad():
    out = torch.softmax(model(X), dim=1)
print(out)

tensor([[0.3113, 0.3934, 0.2952]])


In [27]:
# 设置高效的数据加载器
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
Y_train = torch.tensor([0,0,0,1,1])

In [28]:
X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6]
])
Y_test = torch.tensor([0,1])

In [29]:
from torch.utils.data import DataLoader, Dataset


class ToyDataset(Dataset):
    def __init__(self, X, Y):
        self.features = X
        self.labels = Y

    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y

    def __len__(self):
        return self.labels.shape[0]

train_ds = ToyDataset(X_train, Y_train)
test_ds = ToyDataset(X_test, Y_test)

In [30]:
len(train_ds)

5

In [31]:
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0
)


test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

In [32]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}", x, y)

Batch 1 tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2 tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 3 tensor([[ 2.7000, -1.5000]]) tensor([1])


In [33]:
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

In [34]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}", x, y)

Batch 1 tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 2 tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])


In [36]:
# A.7 典型的训练循环

import torch.nn.functional as F

torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#         日志
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx+1:03d}/{len(train_loader):03d}"
              f" | Train/Val Loss: {loss:.2f}")
model.eval()


Epoch: 001/003 | Batch 001/002 | Train/Val Loss: 0.75
Epoch: 001/003 | Batch 002/002 | Train/Val Loss: 0.65
Epoch: 002/003 | Batch 001/002 | Train/Val Loss: 0.44
Epoch: 002/003 | Batch 002/002 | Train/Val Loss: 0.13
Epoch: 003/003 | Batch 001/002 | Train/Val Loss: 0.03
Epoch: 003/003 | Batch 002/002 | Train/Val Loss: 0.00


NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=2, bias=True)
  )
)

In [44]:
with torch.no_grad():
    outputs = model(X_train)
print(outputs)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])


In [45]:
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

predictions = torch.argmax(probas,dim=1)
print(predictions)

tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])
tensor([0, 0, 0, 1, 1])


In [46]:
predictions = torch.argmax(outputs, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


In [47]:
predictions == Y_train

tensor([True, True, True, True, True])

In [48]:
torch.sum(predictions == Y_train)

tensor(5)

In [50]:
def compute_accuracy(model, dataloader):
    model = model.eval()
    correct = 0
    total_examples = 0

    for idx, (features, labels) in enumerate(dataloader):
        with torch.no_grad():
            logits = model(features)
        predictions = torch.argmax(logits, dim=1)
        compare = labels == predictions
        correct += torch.sum(compare)
        total_examples += len(compare)
    return (correct / total_examples).item()

In [51]:
compute_accuracy(model, train_loader)

1.0

In [52]:
compute_accuracy(model, test_loader)

1.0

In [ ]:
# A.8 保存和加载模型

In [53]:
torch.save(model.state_dict(), "model.pth")



In [57]:
model = NeuralNetwork(2,2)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>